In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
import time

In [2]:
year = 2022
url = f"https://www.pro-football-reference.com/years/{year}/coaches.htm" #can expand to other years later
response = requests.get(url)
if response.status_code == requests.codes.ok:
    soup = BeautifulSoup(response.text, features="html.parser")
else:
    response.raise_for_status()

table = soup.find('table', {'id':'coaches'})
coach_data = (
    pd.read_html(str(table))[0]
    .iloc[1:]
)
coach_data['year'] = year

C:\Users\irela\AppData\Local\Temp\ipykernel_26592\3558236294.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(table))[0]


In [3]:
def retrieve_year_coach_data(year):
    url = f"https://www.pro-football-reference.com/years/{year}/coaches.htm"
    response = requests.get(url)
    if response.status_code == requests.codes.ok:
        soup = BeautifulSoup(response.text, features="html.parser")
    else:
        response.raise_for_status()

    table = soup.find('table', {'id':'coaches'})
    coach_data = (
        pd.read_html(StringIO(str(table)))[0]
        .iloc[1:]
    )
    coach_data['year'] = year
    return coach_data

In [4]:
def clean_coach_data_year(year):
    df = retrieve_year_coach_data(year)
    df.columns = df.columns.droplevel(0)
    df = (
        df[['Coach', 'Tm', '', 'G']]
        .rename(columns={'':'year'})
        .iloc[:, 0:4]
        .rename(columns={'Coach':'coach','Tm':'team', 'G':'num_games_coached'})
    )
    df = df.loc[df.groupby('team')['num_games_coached'].idxmax(), ['team', 'coach', 'year', 'num_games_coached']].reset_index(drop=True)
    return df

In [5]:
def get_all_coach_data(begin_range = 1999, end_range=2025):
    data_list = []
    years = range(begin_range, end_range)
    for year in years:
        data_list.append(clean_coach_data_year(year))
    return pd.concat(data_list)

In [6]:
all_coach_data = get_all_coach_data()

In [8]:
#coach_2025 = get_all_coach_data(begin_range=2024, end_range=2025)

In [10]:
all_coach_data = pd.concat([all_coach_data, coach_2025])
all_coach_data.tail()

,team,coach,year,num_games_coached
26,SEA,Mike Macdonald,2024,17
27,SFO,Kyle Shanahan,2024,17
28,TAM,Todd Bowles,2024,17
29,TEN,Brian Callahan,2024,17
30,WAS,Dan Quinn,2024,17


In [12]:
all_coach_data.to_csv('../data/all_coach_data.csv')